In [59]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelBinarizer

%matplotlib inline

In [60]:
# Load the Wine Dataset
df_wine = pd.read_csv("../Country_vertical/df_wine_vertical.csv", encoding = 'utf8', index_col=0)

In [61]:
df_wine.head(1)

,country,province,region,price,variety,points,year_of_wine
0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013


# Data Preparition

In [62]:
# DEFINE X and Y VARIABLES
df_X = df_wine.drop('points', axis=1)
df_Y = df_wine[['points']]

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0)

In [63]:
# GROUND TRUTH
gt_test_training = pd.concat([X_test, X_train])
gt_training = X_train.copy()
gt_test = X_test.copy()


# INTRODUCE MISSING VALUES

In [64]:
def add_missing_values(df, column_name, ratio):
    ix = [(row, col) for row in range(df[column_name].shape[0]) for col in range(df.shape[1])]
    for row, col in random.sample(ix, int(round(ratio*len(ix)))):
        if is_string_dtype(df[column_name]):
            df.iat[row, col] = 'X'
        elif is_numeric_dtype(df[column_name]):
            df.iat[row, col] = 0
    
def dtr(X_train, y_train, X_test):
    # Decision Tree Regressor
    regressor = DecisionTreeRegressor()  
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    
    # The evaluation metrics
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)*100
    
    results = {"MAE" : round(MAE, 3),
              "MSE" : round(MSE, 3),
              "RMSE" : round(RMSE, 3),
              "R2": round(R2, 3)}
    return results

In [65]:
# DECISION TREE WITHOUT MISSING VALUES
print('BASE RESULTS:')
X_test = pd.get_dummies(X_test, columns=['country'])
X_test = pd.get_dummies(X_test, columns=['province'])
X_test = pd.get_dummies(X_test, columns=['variety'])
X_test = pd.get_dummies(X_test, columns=['region'])

X_train = pd.get_dummies(X_train, columns=['country'])
X_train = pd.get_dummies(X_train, columns=['province'])
X_train = pd.get_dummies(X_train, columns=['variety'])
X_train = pd.get_dummies(X_train, columns=['region'])
    
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
print(dtr(X_train, y_train, X_test))
X_train = gt_training
X_test = gt_test

BASE RESULTS:
{'MAE': 1.987, 'MSE': 7.172, 'RMSE': 2.678, 'R2': 23.814}


In [56]:
# MISSING VALUES IN TEST
ratios = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
results_all_missing_in_test = {}
for ratio in ratios:
    add_missing_values(X_test[['country']], 'country', ratio)
    add_missing_values(X_test[['region']], 'region', ratio)
    add_missing_values(X_test[['variety']], 'variety', ratio)
    add_missing_values(X_test[['province']], 'province', ratio)
    add_missing_values(X_test[['price']], 'price', ratio)
    add_missing_values(X_test[['year_of_wine']], 'year_of_wine', ratio)

    X_test = pd.get_dummies(X_test, columns=['country'])
    X_test = pd.get_dummies(X_test, columns=['province'])
    X_test = pd.get_dummies(X_test, columns=['variety'])
    X_test = pd.get_dummies(X_test, columns=['region'])

    X_train = pd.get_dummies(X_train, columns=['country'])
    X_train = pd.get_dummies(X_train, columns=['province'])
    X_train = pd.get_dummies(X_train, columns=['variety'])
    X_train = pd.get_dummies(X_train, columns=['region'])
    
    X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
    
    results_all_missing_in_test[ratio] = dtr(X_train, y_train, X_test)
    
    X_train = gt_training
    X_test = gt_test

In [9]:
results_all_missing_in_test

{0.1: {'MAE': 1.988, 'MSE': 7.174, 'RMSE': 2.678, 'R2': 23.79},
 0.2: {'MAE': 1.986, 'MSE': 7.133, 'RMSE': 2.671, 'R2': 24.229},
 0.3: {'MAE': 1.986, 'MSE': 7.154, 'RMSE': 2.675, 'R2': 24.001},
 0.4: {'MAE': 1.985, 'MSE': 7.147, 'RMSE': 2.673, 'R2': 24.082},
 0.5: {'MAE': 1.987, 'MSE': 7.164, 'RMSE': 2.677, 'R2': 23.9},
 0.6: {'MAE': 1.989, 'MSE': 7.163, 'RMSE': 2.676, 'R2': 23.908},
 0.7: {'MAE': 1.99, 'MSE': 7.18, 'RMSE': 2.68, 'R2': 23.731},
 0.8: {'MAE': 1.983, 'MSE': 7.147, 'RMSE': 2.673, 'R2': 24.076},
 0.9: {'MAE': 1.986, 'MSE': 7.167, 'RMSE': 2.677, 'R2': 23.867}}

In [10]:
# MISSING VALUES IN TRAIN
ratios = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
results_all_missing_in_train = {}
for ratio in ratios:
    add_missing_values(X_train[['country']], 'country', ratio)
    add_missing_values(X_train[['region']], 'region', ratio)
    add_missing_values(X_train[['variety']], 'variety', ratio)
    add_missing_values(X_train[['province']], 'province', ratio)
    add_missing_values(X_train[['price']], 'price', ratio)
    add_missing_values(X_train[['year_of_wine']], 'year_of_wine', ratio)

    X_test = pd.get_dummies(X_test, columns=['country'])
    X_test = pd.get_dummies(X_test, columns=['province'])
    X_test = pd.get_dummies(X_test, columns=['variety'])
    X_test = pd.get_dummies(X_test, columns=['region'])

    X_train = pd.get_dummies(X_train, columns=['country'])
    X_train = pd.get_dummies(X_train, columns=['province'])
    X_train = pd.get_dummies(X_train, columns=['variety'])
    X_train = pd.get_dummies(X_train, columns=['region'])
    
    X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
    
    results_all_missing_in_train[ratio] = dtr(X_train, y_train, X_test)
    
    X_train = gt_training
    X_test = gt_test

In [11]:
results_all_missing_in_train

{0.1: {'MAE': 1.988, 'MSE': 7.17, 'RMSE': 2.678, 'R2': 23.838},
 0.2: {'MAE': 1.988, 'MSE': 7.175, 'RMSE': 2.679, 'R2': 23.783},
 0.3: {'MAE': 1.982, 'MSE': 7.13, 'RMSE': 2.67, 'R2': 24.266},
 0.4: {'MAE': 1.986, 'MSE': 7.16, 'RMSE': 2.676, 'R2': 23.944},
 0.5: {'MAE': 1.987, 'MSE': 7.169, 'RMSE': 2.677, 'R2': 23.849},
 0.6: {'MAE': 1.988, 'MSE': 7.162, 'RMSE': 2.676, 'R2': 23.915},
 0.7: {'MAE': 1.982, 'MSE': 7.145, 'RMSE': 2.673, 'R2': 24.106},
 0.8: {'MAE': 1.986, 'MSE': 7.155, 'RMSE': 2.675, 'R2': 23.991},
 0.9: {'MAE': 1.984, 'MSE': 7.146, 'RMSE': 2.673, 'R2': 24.087}}

In [369]:
len(df_wine.country.unique().tolist())

7

# Precision & Recall

In [132]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(gt_test_training['country'].values, 
                                dirty_test_training['country'].values, 
                                average='weighted')

NameError: name 'dirty_test_training' is not defined

# HPO with ground truth

In [133]:
# # Using Randomized Search for hyperparameter optimisation

# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from scipy.stats import randint as sp_randint
# regressor3 = DecisionTreeRegressor()
# param_dist = {'max_depth': sp_randint(2,16),
#               'min_samples_split': sp_randint(2,16)}

# n_iter_search = 20
# clfrs = RandomizedSearchCV(regressor3, 
#                                    param_distributions=param_dist,
#                                    scoring='neg_mean_squared_error',
#                                    cv=5 , n_jobs=1, verbose=1,
#                                    n_iter = n_iter_search)
# clfrs.fit(df_X, df_Y)
# clfrs.best_params_, clfrs.best_score_